<a href="https://colab.research.google.com/github/kkhhmm3103/SSU_Datathon2025/blob/main/BERT%E1%84%8C%E1%85%A5%E1%86%AB%E1%84%8E%E1%85%A5%E1%84%85%E1%85%B5_%E1%84%8C%E1%85%A6%E1%84%8E%E1%85%AE%E1%86%AF%E1%84%8B%E1%85%AD%E1%86%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## 라이브러리
import json
import pandas as pd
import re
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from konlpy.tag import Okt
from collections import Counter

In [ ]:
# 파일 경로 (raw string 사용)
file_path = r"SSU_Datathon2025_공학분야_62199.json"

# JSON 로드
with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# NODE_LIST를 DataFrame으로
df = pd.DataFrame(data["NODE_LIST"])

# 확인
print(df.head())
print(df.info())

        NODE_ID    IPRD_NM        PLCT_NM  \
0  NODE10561230  제어로봇시스템학회  제어로봇시스템학회 논문지   
1  NODE10561231  제어로봇시스템학회  제어로봇시스템학회 논문지   
2  NODE10561232  제어로봇시스템학회  제어로봇시스템학회 논문지   
3  NODE10561237  제어로봇시스템학회  제어로봇시스템학회 논문지   
4  NODE10561238  제어로봇시스템학회  제어로봇시스템학회 논문지   

                                           NODE_TTLE  \
0                    카테시안 공간에서 시간 지연제어기의 구현에 대한 튜토리얼   
1                    파이프 구조물 검사를 위한 파이프 등반 로봇의 위치 추적   
2  Depth와 Amplitude Image를 이용한 차량 문 주변 장애물 인식 및 거...   
3    단시간 푸리에 변환과 전역 통과 필터 기반 노치 필터를 이용한 다중 주파수 제거 기법   
4                    MEMS/FOG 기반 관성항법장치 2단계 교정 방법 개발   

                                        NODE_TTLE_EN    PBSH  \
0  Tutorial on Implementation of Time-delayed Con...  202106   
1  Position Tracking of Pipe-Climbing Robot for P...  202106   
2  Obstacle Recognition and Distance Estimation n...  202106   
3  Multiple Frequency Rejection using Short-time ...  202106   
4  Development of Two-Stage Calibration Technique...  202106   

       

In [ ]:
def has_text(x):
    return isinstance(x, str) and x.strip() != ""

# 삭제 대상: 한글 초록도 없고(ABST_KR), 영어 초록도 없는(ABST_EN) 행
mask_no_abstract = (
    ~df["ABST_KR"].apply(has_text) &
    ~df["ABST_EN"].apply(has_text)
)

print("전체 행 수:", len(df))
print("초록(한/영) 둘 다 없는 행 수(삭제 대상):", int(mask_no_abstract.sum()))

# 초록 없는 행 삭제
df_no_empty_abstract = df.loc[~mask_no_abstract].copy()

print("삭제 후 남은 행 수:", len(df_no_empty_abstract))

# 저장
out_path = "SSU_Datathon2025_공학분야_62199_no_empty_abstract.csv"
df_no_empty_abstract.to_csv(out_path, index=False, encoding="utf-8-sig")
print("저장 완료:", out_path)

전체 행 수: 62199
초록(한/영) 둘 다 없는 행 수(삭제 대상): 15151
삭제 후 남은 행 수: 47048
저장 완료: SSU_Datathon2025_공학분야_62199_no_empty_abstract.csv


In [ ]:
# 입력 CSV (초록 없는 행 제거된 파일)
in_path = "SSU_Datathon2025_공학분야_62199_no_empty_abstract.csv"

# 출력 CSV (키워드 없는 행만)
out_path = "SSU_Datathon2025_공학분야_62199_no_keyword.csv"

# CSV 로드
df = pd.read_csv(in_path)

def has_text(x):
    return isinstance(x, str) and x.strip() != ""

# 키워드가 없는 행
mask_no_keyword = ~df["KYWD"].apply(has_text)

df_no_keyword = df.loc[mask_no_keyword].copy()

# 개수 확인
print("전체 행 수:", len(df))
print("키워드가 없는 행 수 (BERT 대상):", len(df_no_keyword))
print("비율:", len(df_no_keyword) / len(df))

# CSV 저장
df_no_keyword.to_csv(
    out_path,
    index=False,
    encoding="utf-8-sig"
)

print("저장 완료:", out_path)

전체 행 수: 47048
키워드가 없는 행 수 (BERT 대상): 1322
비율: 0.02809896276143513
저장 완료: SSU_Datathon2025_공학분야_62199_no_keyword.csv


In [ ]:
# 초록 없고 키워드 없는 행 제거된 CSV
in_path = "SSU_Datathon2025_공학분야_62199_no_keyword.csv"
df = pd.read_csv(in_path)

def has_text(x):
    return isinstance(x, str) and x.strip() != ""

# 컬럼명
KR_TITLE = "NODE_TTLE"
EN_TITLE = "NODE_TTLE_EN"
KR_ABST  = "ABST_KR"
EN_ABST  = "ABST_EN"

# ===== 제목 언어 상태 =====
has_kr_title = df[KR_TITLE].apply(has_text)
has_en_title = df[EN_TITLE].apply(has_text)

title_state = pd.Series(index=df.index, dtype="object")

title_state[ has_kr_title & ~has_en_title] = "한글 제목만"
title_state[~has_kr_title &  has_en_title] = "영어 제목만"
title_state[ has_kr_title &  has_en_title] = "한글+영어 제목"
title_state[~has_kr_title & ~has_en_title] = "제목 없음"

print("제목 언어 분포")
print(title_state.value_counts())
print("\n비율(%)")
print((title_state.value_counts() / len(df) * 100).round(2))

# ===== 초록 언어 상태 =====
has_kr_abst = df[KR_ABST].apply(has_text)
has_en_abst = df[EN_ABST].apply(has_text)

abst_state = pd.Series(index=df.index, dtype="object")

abst_state[ has_kr_abst & ~has_en_abst] = "한글 초록만"
abst_state[~has_kr_abst &  has_en_abst] = "영어 초록만"
abst_state[ has_kr_abst &  has_en_abst] = "한글+영어 초록"

print("\n초록 언어 분포")
print(abst_state.value_counts())
print("\n비율(%)")
print((abst_state.value_counts() / len(df) * 100).round(2))

📌 제목 언어 분포
한글 제목만      964
한글+영어 제목    358
Name: count, dtype: int64

비율(%)
한글 제목만      72.92
한글+영어 제목    27.08
Name: count, dtype: float64

📌 초록 언어 분포
한글 초록만      1022
한글+영어 초록     256
영어 초록만        44
Name: count, dtype: int64

비율(%)
한글 초록만      77.31
한글+영어 초록    19.36
영어 초록만       3.33
Name: count, dtype: float64


In [ ]:
# 입력 CSV
in_path = "SSU_Datathon2025_공학분야_62199_no_keyword.csv"
df = pd.read_csv(in_path)

def has_text(x):
    return isinstance(x, str) and x.strip() != ""

# 컬럼명
KR_TITLE = "NODE_TTLE"
EN_TITLE = "NODE_TTLE_EN"
KR_ABST  = "ABST_KR"
EN_ABST  = "ABST_EN"

# Boolean 마스크
has_kr_title = df[KR_TITLE].apply(has_text)
has_en_title = df[EN_TITLE].apply(has_text)
has_kr_abst  = df[KR_ABST].apply(has_text)
has_en_abst  = df[EN_ABST].apply(has_text)

# 한글 초록만 있는 경우
mask_kr_only = has_kr_abst & ~has_en_abst

kr_only_total = mask_kr_only.sum()
kr_only_with_kr_title = (mask_kr_only & has_kr_title).sum()
kr_only_without_kr_title = kr_only_total - kr_only_with_kr_title

# 영어 초록만 있는 경우
mask_en_only = has_en_abst & ~has_kr_abst

en_only_total = mask_en_only.sum()
en_only_with_en_title = (mask_en_only & has_en_title).sum()
en_only_without_en_title = en_only_total - en_only_with_en_title

# 두 언어 초록이 있는 경우
mask_both_abst = has_kr_abst & has_en_abst

both_total = mask_both_abst.sum()
both_with_both_titles = (mask_both_abst & has_kr_title & has_en_title).sum()
both_with_kr_title_only = (mask_both_abst & has_kr_title & ~has_en_title).sum()
both_with_en_title_only = (mask_both_abst & ~has_kr_title & has_en_title).sum()
both_with_no_title = (
    mask_both_abst & ~has_kr_title & ~has_en_title
).sum()

# 결과 출력
print("[1] 한글 초록만 있는 경우")
print("총 개수:", kr_only_total)
print(" ├ 한글 제목 있음:", kr_only_with_kr_title)
print(" └ 한글 제목 없음:", kr_only_without_kr_title)

print("\n[2] 영어 초록만 있는 경우")
print("총 개수:", en_only_total)
print(" ├ 영어 제목 있음:", en_only_with_en_title)
print(" └ 영어 제목 없음:", en_only_without_en_title)

print("\n[3] 한글+영어 초록 모두 있는 경우")
print("총 개수:", both_total)
print(" ├ 한글+영어 제목 모두 있음:", both_with_both_titles)
print(" ├ 한글 제목만 있음:", both_with_kr_title_only)
print(" ├ 영어 제목만 있음:", both_with_en_title_only)
print(" └ 제목 없음:", both_with_no_title)


📌 [1] 한글 초록만 있는 경우
총 개수: 1022
 ├ 한글 제목 있음: 1022
 └ 한글 제목 없음: 0

📌 [2] 영어 초록만 있는 경우
총 개수: 44
 ├ 영어 제목 있음: 42
 └ 영어 제목 없음: 2

📌 [3] 한글+영어 초록 모두 있는 경우
총 개수: 256
 ├ 한글+영어 제목 모두 있음: 247
 ├ 한글 제목만 있음: 9
 ├ 영어 제목만 있음: 0
 └ 제목 없음: 0


In [ ]:
def has_text(x):
    return isinstance(x, str) and x.strip() != ""

KR_TITLE = "NODE_TTLE"
EN_TITLE = "NODE_TTLE_EN"
KR_ABST  = "ABST_KR"
EN_ABST  = "ABST_EN"

# 조건:
# 1) 영어 초록 있음
# 2) 한글 초록 없음
# 3) 영어 제목 없음
mask_en_abs_no_en_title = (
    df[EN_ABST].apply(has_text) &
    ~df[KR_ABST].apply(has_text) &
    ~df[EN_TITLE].apply(has_text)
)

df_en_abs_no_en_title = df.loc[mask_en_abs_no_en_title].copy()

print("영어 초록만 있고 영어 제목 없는 행 개수:", len(df_en_abs_no_en_title))

# 핵심 컬럼만 출력
cols_to_show = [
    "NODE_ID",
    "NODE_TTLE",
    "NODE_TTLE_EN",
    "ABST_EN",
    "KYWD"
]

print(df_en_abs_no_en_title[cols_to_show].head(10))

영어 초록만 있고 영어 제목 없는 행 개수: 2
          NODE_ID                                          NODE_TTLE  \
345  NODE11124753                                          6G 미래기술동향   
624  NODE11763742  MPCitH 기반 영지식 증명과 대칭키 프리미티브 기반 일방향 함수를 결합 양자 내...   

    NODE_TTLE_EN                                            ABST_EN  KYWD  
345          NaN  International Telecommunication Union-Radiocom...   NaN  
624          NaN  양자 컴퓨팅 환경을 대비한 암호 시스템에 대한 중요성이 대두되면서 국내외로 양자 내...   NaN  


In [ ]:
def has_text(x):
    return isinstance(x, str) and x.strip() != ""

KR_TITLE = "NODE_TTLE"
EN_TITLE = "NODE_TTLE_EN"
KR_ABST  = "ABST_KR"
EN_ABST  = "ABST_EN"
ID_COL   = "NODE_ID"

# 교정 대상: 624에 해당하는 NODE_ID
target_id = "NODE11763742"

mask_target = (df[ID_COL] == target_id)

print("타겟 행 수:", int(mask_target.sum()))

cols_check = [ID_COL, KR_TITLE, EN_TITLE, KR_ABST, EN_ABST, "KYWD"]
print("\n[수정 전]")
print(df.loc[mask_target, cols_check].head(1))

# - ABST_KR 비어있고
# - ABST_EN는 텍스트가 있을 때만
mask_fix = mask_target & ~df[KR_ABST].apply(has_text) & df[EN_ABST].apply(has_text)

print("\n실제 교정 적용 행 수:", int(mask_fix.sum()))

# EN → KR로 이동, EN 비우기
df_fixed = df.copy()
df_fixed.loc[mask_fix, KR_ABST] = df_fixed.loc[mask_fix, EN_ABST]
df_fixed.loc[mask_fix, EN_ABST] = ""

print("\n[수정 후]")
print(df_fixed.loc[mask_target, cols_check].head(1))

# 새 CSV 저장
out_path = "SSU_Datathon2025_공학분야_62199_no_empty_abstract_fix624.csv"
df_fixed.to_csv(out_path, index=False, encoding="utf-8-sig")
print("\n저장 완료:", out_path)

타겟 행 수: 1

[수정 전]
          NODE_ID                                          NODE_TTLE  \
624  NODE11763742  MPCitH 기반 영지식 증명과 대칭키 프리미티브 기반 일방향 함수를 결합 양자 내...   

    NODE_TTLE_EN ABST_KR                                            ABST_EN  \
624          NaN     NaN  양자 컴퓨팅 환경을 대비한 암호 시스템에 대한 중요성이 대두되면서 국내외로 양자 내...   

     KYWD  
624   NaN  

실제 교정 적용 행 수: 1

[수정 후]
          NODE_ID                                          NODE_TTLE  \
624  NODE11763742  MPCitH 기반 영지식 증명과 대칭키 프리미티브 기반 일방향 함수를 결합 양자 내...   

    NODE_TTLE_EN                                            ABST_KR ABST_EN  \
624          NaN  양자 컴퓨팅 환경을 대비한 암호 시스템에 대한 중요성이 대두되면서 국내외로 양자 내...           

     KYWD  
624   NaN  

저장 완료: SSU_Datathon2025_공학분야_62199_no_empty_abstract_fix624.csv


In [ ]:
# 입력 CSV
in_path = "SSU_Datathon2025_공학분야_62199_no_empty_abstract_fix624.csv"
df = pd.read_csv(in_path)

def has_text(x):
    return isinstance(x, str) and x.strip() != ""

# 컬럼명
KR_TITLE = "NODE_TTLE"
EN_TITLE = "NODE_TTLE_EN"
KR_ABST  = "ABST_KR"
EN_ABST  = "ABST_EN"

# Boolean 마스크
has_kr_title = df[KR_TITLE].apply(has_text)
has_en_title = df[EN_TITLE].apply(has_text)
has_kr_abst  = df[KR_ABST].apply(has_text)
has_en_abst  = df[EN_ABST].apply(has_text)

# 한글 초록만 있는 경우
mask_kr_only = has_kr_abst & ~has_en_abst

kr_only_total = mask_kr_only.sum()
kr_only_with_kr_title = (mask_kr_only & has_kr_title).sum()
kr_only_without_kr_title = kr_only_total - kr_only_with_kr_title

# 영어 초록만 있는 경우
mask_en_only = has_en_abst & ~has_kr_abst

en_only_total = mask_en_only.sum()
en_only_with_en_title = (mask_en_only & has_en_title).sum()
en_only_without_en_title = en_only_total - en_only_with_en_title

# 두 언어 초록이 있는 경우
mask_both_abst = has_kr_abst & has_en_abst

both_total = mask_both_abst.sum()
both_with_both_titles = (mask_both_abst & has_kr_title & has_en_title).sum()
both_with_kr_title_only = (mask_both_abst & has_kr_title & ~has_en_title).sum()
both_with_en_title_only = (mask_both_abst & ~has_kr_title & has_en_title).sum()
both_with_no_title = (
    mask_both_abst & ~has_kr_title & ~has_en_title
).sum()

print("[1] 한글 초록만 있는 경우")
print("총 개수:", kr_only_total)
print(" ├ 한글 제목 있음:", kr_only_with_kr_title)
print(" └ 한글 제목 없음:", kr_only_without_kr_title)

print("\n[2] 영어 초록만 있는 경우")
print("총 개수:", en_only_total)
print(" ├ 영어 제목 있음:", en_only_with_en_title)
print(" └ 영어 제목 없음:", en_only_without_en_title)

print("\n[3] 한글+영어 초록 모두 있는 경우")
print("총 개수:", both_total)
print(" ├ 한글+영어 제목 모두 있음:", both_with_both_titles)
print(" ├ 한글 제목만 있음:", both_with_kr_title_only)
print(" ├ 영어 제목만 있음:", both_with_en_title_only)
print(" └ 제목 없음:", both_with_no_title)

📌 [1] 한글 초록만 있는 경우
총 개수: 1023
 ├ 한글 제목 있음: 1023
 └ 한글 제목 없음: 0

📌 [2] 영어 초록만 있는 경우
총 개수: 43
 ├ 영어 제목 있음: 42
 └ 영어 제목 없음: 1

📌 [3] 한글+영어 초록 모두 있는 경우
총 개수: 256
 ├ 한글+영어 제목 모두 있음: 247
 ├ 한글 제목만 있음: 9
 ├ 영어 제목만 있음: 0
 └ 제목 없음: 0


In [ ]:
def has_text(x):
    return isinstance(x, str) and x.strip() != ""

# (1) 입력 파일: 624 교정까지 반영한 파일을 쓰는 걸 추천
in_path = "SSU_Datathon2025_공학분야_62199_no_empty_abstract_fix624.csv"

# (2) 출력 파일
out_path = "SSU_Datathon2025_공학분야_62199_input_text_ready.csv"

df = pd.read_csv(in_path)

# 컬럼명
KR_TITLE = "NODE_TTLE"
EN_TITLE = "NODE_TTLE_EN"
KR_ABST  = "ABST_KR"
EN_ABST  = "ABST_EN"

# 마스크
has_kr_abst = df[KR_ABST].apply(has_text)
has_en_abst = df[EN_ABST].apply(has_text)
has_kr_title = df[KR_TITLE].apply(has_text)
has_en_title = df[EN_TITLE].apply(has_text)

mask_kr_only  = has_kr_abst & ~has_en_abst
mask_en_only  = has_en_abst & ~has_kr_abst
mask_both_abs = has_kr_abst & has_en_abst

# 예외: 영어 초록만 있는데 영어 제목 없음
mask_en_only_no_en_title = mask_en_only & ~has_en_title

def build_input_text(row):
    kr_title = row.get(KR_TITLE, "")
    en_title = row.get(EN_TITLE, "")
    kr_abst  = row.get(KR_ABST, "")
    en_abst  = row.get(EN_ABST, "")

    # 한글+영어 초록 둘 다 있는 경우 -> 한글 제목 + 한글 초록
    if has_text(kr_abst) and has_text(en_abst):
        if has_text(kr_title):
            return f"{kr_title}\n{kr_abst}"
        return str(kr_abst).strip()

    # 한글 초록만 있는 경우 -> 한글 제목 + 한글 초록
    if has_text(kr_abst) and not has_text(en_abst):
        if has_text(kr_title):
            return f"{kr_title}\n{kr_abst}"
        return str(kr_abst).strip()

    # 영어 초록만 있는 경우 -> 영어 제목 + 영어 초록
    if has_text(en_abst) and not has_text(kr_abst):
        # 4) 예외: 영어 제목 없으면 영어 초록만
        if not has_text(en_title):
            return str(en_abst).strip()
        return f"{en_title}\n{en_abst}"

    return ""

def classify_case(row):
    kr_title = row.get(KR_TITLE, "")
    en_title = row.get(EN_TITLE, "")
    kr_abst  = row.get(KR_ABST, "")
    en_abst  = row.get(EN_ABST, "")

    if has_text(kr_abst) and has_text(en_abst):
        return "CASE3_KR+EN_abs_use_KR"
    if has_text(kr_abst) and not has_text(en_abst):
        return "CASE1_KR_only"
    if has_text(en_abst) and not has_text(kr_abst):
        if not has_text(en_title):
            return "CASE4_EN_only_no_EN_title"
        return "CASE2_EN_only"
    return "UNKNOWN"

def input_lang(row):
    kr_abst = row.get(KR_ABST, "")
    en_abst = row.get(EN_ABST, "")
    en_title = row.get(EN_TITLE, "")

    if has_text(kr_abst):
        return "KR"  # KR-only 또는 KR+EN에서 KR 사용
    if has_text(en_abst) and has_text(en_title):
        return "EN"
    if has_text(en_abst) and not has_text(en_title):
        return "EN"  # 예외도 EN 초록만 사용
    return ""

# INPUT_TEXT 생성
df["INPUT_TEXT"] = df.apply(build_input_text, axis=1)

df["INPUT_CASE"] = df.apply(classify_case, axis=1)
df["INPUT_LANG"] = df.apply(input_lang, axis=1)

# sanity check
print("전체 행 수:", len(df))
print("INPUT_TEXT 비어있는 행 수:", int((df["INPUT_TEXT"].apply(lambda x: not has_text(x))).sum()))
print("\n[케이스 분포]")
print(df["INPUT_CASE"].value_counts())

# 저장
df.to_csv(out_path, index=False, encoding="utf-8-sig")
print("\n저장 완료:", out_path)

전체 행 수: 1322
INPUT_TEXT 비어있는 행 수: 0

[케이스 분포]
INPUT_CASE
CASE1_KR_only                1023
CASE3_KR+EN_abs_use_KR        256
CASE2_EN_only                  42
CASE4_EN_only_no_EN_title       1
Name: count, dtype: int64

저장 완료: SSU_Datathon2025_공학분야_62199_input_text_ready.csv


In [ ]:
in_path  = "SSU_Datathon2025_공학분야_62199_input_text_ready.csv"
out_path = "SSU_Datathon2025_공학분야_62199_kw_filled_ko_en_models.csv"

df = pd.read_csv(in_path)

def has_text(x):
    return isinstance(x, str) and x.strip() != ""

# 모델 설정
KO_MODEL_NAME = "snunlp/KR-SBERT-V40K-klueNLI-augSTS"
EN_MODEL_NAME = "sentence-transformers/all-mpnet-base-v2"

ko_model = SentenceTransformer(KO_MODEL_NAME)
en_model = SentenceTransformer(EN_MODEL_NAME)

# 하이퍼파라미터
TOP_K = 6                 # 문서당 최종 키워드 수
CANDIDATE_LIMIT = 28      # 후보 상한
NGRAM_MAX = 3             # 한국어 명사구 최대 길이
OUT_SEP = ", "

okt = Okt()

# 후보 생성 - 한국어
def make_candidates_kr(text, limit=CANDIDATE_LIMIT):
    if not has_text(text):
        return []

    nouns = [n for n in okt.nouns(text) if len(n) >= 2]
    if not nouns:
        return []

    cands = set(nouns)

    for n in range(2, NGRAM_MAX + 1):
        for i in range(len(nouns) - n + 1):
            phrase = " ".join(nouns[i:i+n])
            if len(phrase.replace(" ", "")) >= 4:
                cands.add(phrase)

    cands = list(cands)
    if len(cands) <= limit:
        return cands

    vec = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b")
    X = vec.fit_transform([text])
    vocab = vec.get_feature_names_out()
    scores = X.toarray().ravel()
    score_map = dict(zip(vocab, scores))

    return sorted(
        cands,
        key=lambda w: score_map.get(w.replace(" ", ""), 0),
        reverse=True
    )[:limit]

# 후보 생성 - 영어
EN_STOP = set("""
a an the and or for to of in on at with without by from into via using use used
is are was were be been being this that these those as such can may might
we our you your their his her its they them
""".split())

def clean_en(text):
    text = (text or "").lower()
    text = re.sub(r"[^a-z0-9\s\-]", " ", text)
    return re.sub(r"\s+", " ", text).strip()

def make_candidates_en(text, limit=CANDIDATE_LIMIT):
    text = clean_en(text)
    if not has_text(text):
        return []

    vec = TfidfVectorizer(
        stop_words=list(EN_STOP),
        ngram_range=(1, 3),
        min_df=1
    )
    X = vec.fit_transform([text])
    vocab = vec.get_feature_names_out()
    scores = X.toarray().ravel()

    pairs = []
    for w, s in zip(vocab, scores):
        if len(w) < 3:
            continue
        if any(tok in EN_STOP for tok in w.split()):
            continue
        pairs.append((w, float(s)))

    pairs.sort(key=lambda x: x[1], reverse=True)
    return [w for w, _ in pairs[:limit]]

# 랭킹
_cache_ko = {}
_cache_en = {}

def embed_candidates(model, cache, candidates, batch_size=64):
    vecs = [None] * len(candidates)
    to_encode, idxs = [], []

    for i, c in enumerate(candidates):
        key = (c or "").strip()
        if not key:
            continue
        if key in cache:
            vecs[i] = cache[key]
        else:
            to_encode.append(key)
            idxs.append(i)

    if to_encode:
        enc = model.encode(
            to_encode,
            batch_size=batch_size,
            show_progress_bar=False,
            normalize_embeddings=True
        )
        for k, v in zip(to_encode, enc):
            cache[k] = v
        for i, k in zip(idxs, to_encode):
            vecs[i] = cache[k]

    return np.vstack([v for v in vecs if v is not None])

def rank_keywords(doc_text, candidates, model, cache):
    if not has_text(doc_text) or not candidates:
        return []

    doc_vec = model.encode([doc_text], normalize_embeddings=True)
    cand_vec = embed_candidates(model, cache, candidates)

    sims = cosine_similarity(doc_vec, cand_vec).ravel()
    idx = np.argsort(-sims)[:TOP_K]
    return [candidates[i] for i in idx]

# 키워드 없는 행만 채우기
KW_COL   = "KYWD"
TEXT_COL = "INPUT_TEXT"
LANG_COL = "INPUT_LANG"

df_out = df.copy()

mask_need = ~df_out[KW_COL].apply(has_text)
print("키워드 채울 대상 수:", int(mask_need.sum()))

def process_row(row):
    text = row.get(TEXT_COL, "")
    lang = row.get(LANG_COL, "")

    if not has_text(text):
        return ""

    if lang == "KR":
        cands = make_candidates_kr(text)
        kws = rank_keywords(text, cands, ko_model, _cache_ko)
        return OUT_SEP.join(kws)

    if lang == "EN":
        cands = make_candidates_en(text)
        kws = rank_keywords(text, cands, en_model, _cache_en)
        return OUT_SEP.join(kws)

    return ""

# 결과 컬럼: 원본 보존 + 채운 결과 별도 기록
df_out["KYWD_FILLED_LANG"] = df_out[KW_COL].astype("string")
df_out["KYWD_IMPUTED_LANG"] = 0

filled = df_out.loc[mask_need].apply(process_row, axis=1).astype("string")

df_out.loc[mask_need, "KYWD_FILLED_LANG"] = filled
df_out.loc[mask_need, "KYWD_IMPUTED_LANG"] = 1

print("채운 후에도 빈 키워드 행 수:",
      int((~df_out["KYWD_FILLED_LANG"].apply(has_text)).sum()))

df_out.to_csv(out_path, index=False, encoding="utf-8-sig")
print("저장 완료:", out_path)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

키워드 채울 대상 수: 1322
채운 후에도 빈 키워드 행 수: 0
저장 완료: SSU_Datathon2025_공학분야_62199_kw_filled_ko_en_models.csv


In [ ]:
main_path = "SSU_Datathon2025_공학분야_62199_no_empty_abstract.csv"
filled_path = "SSU_Datathon2025_공학분야_62199_kw_filled_ko_en_models.csv"
out_path = "keyword_bert_df.csv"

df_main = pd.read_csv(main_path)
df_filled = pd.read_csv(filled_path)

# 필수 컬럼 존재 확인
required_main = {"NODE_ID", "KYWD"}
required_filled = {"NODE_ID", "KYWD_FILLED_LANG"}

missing_main = required_main - set(df_main.columns)
missing_filled = required_filled - set(df_filled.columns)

if missing_main:
    raise ValueError(f"메인 파일에 필요한 컬럼이 없습니다: {missing_main}")
if missing_filled:
    raise ValueError(f"filled 파일에 필요한 컬럼이 없습니다: {missing_filled}")

# NODE_ID 중복이 있으면 마지막 값으로 정리
df_filled = df_filled.dropna(subset=["NODE_ID"])
df_filled = df_filled.drop_duplicates(subset=["NODE_ID"], keep="last")

# 메인 파일에서 비어있는 KYWD 판별
# - NaN 이거나
# - 공백/빈문자열 이거나
# - "nan", "None" 같은 문자열로 들어온 경우도 방어
def is_empty_kw(s):
    if pd.isna(s):
        return True
    s2 = str(s).strip()
    return (s2 == "") or (s2.lower() in {"nan", "none", "null"})

empty_mask = df_main["KYWD"].apply(is_empty_kw)

# NODE_ID -> keyword_filled_lang 매핑 딕셔너리 생성
kw_map = dict(zip(df_filled["NODE_ID"], df_filled["KYWD_FILLED_LANG"]))

# 비어있는 KYWD만 채우기 (매칭되는 NODE_ID에 한해서)
before_empty = empty_mask.sum()

df_main.loc[empty_mask, "KYWD"] = df_main.loc[empty_mask, "NODE_ID"].map(kw_map).combine_first(
    df_main.loc[empty_mask, "KYWD"]
)

after_empty = df_main["KYWD"].apply(is_empty_kw).sum()
filled_count = before_empty - after_empty

print("메인 데이터 행 수:", len(df_main))
print("채우기 전 KYWD 비어있던 행 수:", before_empty)
print("이번에 채워진 행 수:", filled_count)
print("채운 후 KYWD 비어있는 행 수:", after_empty)

df_main.to_csv(out_path, index=False, encoding="utf-8-sig")
print("최종 저장 완료:", out_path)

메인 데이터 행 수: 47048
채우기 전 KYWD 비어있던 행 수: 1322
이번에 채워진 행 수: 1322
채운 후 KYWD 비어있는 행 수: 0
최종 저장 완료: SSU_Datathon2025_공학분야_62199_KYWD_filled_all.csv
